# Data Visualization
This notebook contains visualizations for the cherry leaves dataset. All generated plots and files will be saved in the '../out/visualization/' directory.

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Ensure output directory exists
output_dir = '../out/visualization'
os.makedirs(output_dir, exist_ok=True)
print(f'Visualization output directory: {output_dir}')